# Week 3 problems

## Question 1: Fitting a Signal in the Presence of Background (40 points)

### Learning objectives
In this question you will:

- Gain experience in performing $\chi^2$ fits to histogrammed data; explore how the statistical significance of a signal depends on the number of signal events and the signal-to-background ratio


### 1a. 

Physicists often fit for signals in the presence of background. Compared to the previous exercise, we now explore the situation where it is not possible to take data separately in background-only mode. In such cases, the significance of the measured signal depends not only on the number of signal events but also on the amount of background *and* our ability to statistically separate the two.  

In this problem, we will explore fitting signal and background for a very simple case:  The signal is a Gaussian peak centered at $x=10$ with a width $\sigma=1$ and the background is uniformly distributed between $x=0$ and $x=20$.  

The code below generates fake data, allowing you to change both the number of events in the signal and the ratio of signal-to-background.  To make sure that our definition of background does not depend on the fit range, the code below defines the signal-to-background ratio as the ratio number of signal events to the number of background events in a $\pm 2 \sigma$ window around the signal peak.  Here is the code you will use to generate the fake data:

In [21]:
# Write import math
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def makeData( SignalToBackground, nSigEvents, seed=12345):
    """Generates a dataset consisting of nEvents some of which are signal (a Gaussian with a width of 1 centered
    at x=5) and the rest of which is background
    Definition of SignalToBackground:  The ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice)
    
    Parameters
    ==========
     SignalToBackground : float
     Ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice) 
      
    nSigEvents : int
      total number of signal events generated (NOT the number in +- 2 sigma)
      
    seed : int
      seed for the random number generator
      
    Returns
    =======
    data : array
      the measurements of x
    """
    fracOutsideTwoSigma = 4.55e-2
    # 1-fracOutsideTwoSigma is the fraction of the Signal Events within +-2 sigma
    # To get the total number of background events, find the number in +-2 sigma
    # which is 4 units of x and then since the background is flat from 0 to 20
    # multiply by 20/4=5
    
    nBackground = 5*nSigEvents*(1-fracOutsideTwoSigma)/SignalToBackground
    nEvents = nSigEvents+nBackground
    fSig = nSigEvents/nEvents
    
    # Make an array to hold the data (ie the x measurements)
    data = []
    
    # set the random seed.  This will allow us to reproduce the results if we run again
    np.random.seed(seed)
 
    # Retrieve nEvents random numbers that will be used to pick which events are signal
    # and which are background
    n = int(nEvents)
    tests = np.random.uniform(0,1,n)
    bck = np.random.uniform(0,20,n)
    sig = np.random.normal(10,1,n)
    
    count = 0
    for test in tests:
        if(test<fSig):
            data.append(sig[count])
        else:
            data.append(bck[count])
        count+=1
    
    # Loop over the events and pick either signal or background and draw from the appropriate 
    # pdf in each case
    
    # return the data to the user
    return data
    

or alternatively using ROOT (do not execute both cells, just either this or the one above depending on what you're planning to use and delete the other from your notebook to avoid mistakes)

In [69]:
import ROOT

def makeData( SignalToBackground, nSigEvents, seed=12345):
    """Generates a dataset consisting of nEvents some of which are signal (a Gaussian with a width of 1 centered
    at x=5) and the rest of which is background
    Definition of SignalToBackground:  The ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice)
    
    Parameters
    ==========
     SignalToBackground : float
     Ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice) 
      
    nSigEvents : int
      total number of signal events generated (NOT the number in +- 2 sigma)
      
    seed : int
      seed for the random number generator
      
    Returns
    =======
    data : array
      the measurements of x
    """
    fracOutsideTwoSigma = 4.55e-2
    # 1-fracOutsideTwoSigma is the fraction of the Signal Events within +-2 sigma
    # To get the total number of background events, find the number in +-2 sigma
    # which is 4 units of x and then since the background is flat from 0 to 20
    # multiply by 20/4=5
    
    nBackground = 5*nSigEvents*(1-fracOutsideTwoSigma)/SignalToBackground
    nEvents = nSigEvents+nBackground
    fSig = nSigEvents/nEvents
    
    # Make an array to hold the data (ie the x measurements)
    data = []
    
    # set the random seed.  This will allow us to reproduce the results if we run again
    ROOT.gRandom.SetSeed(seed)
 
    # Retrieve nEvents random numbers that will be used to pick which events are signal
    # and which are background
    n = int(nEvents)
    tests = []
    bck = []
    sig = []
    for i in range(n):
        tests.append(ROOT.gRandom.Rndm())
        bck.append(ROOT.gRandom.Rndm()*20.0)
        sig.append(ROOT.gRandom.Gaus(10, 1))
    
    # Loop over the events and pick either signal or background and draw from the appropriate 
    # pdf in each case
    count = 0
    for test in tests:
        if(test<fSig):
            data.append(sig[count])
        else:
            data.append(bck[count])
        count+=1
        
    # return the data to the user
    return data
    

Below is a simple test to show you how to use this code.  Remember if you run this code multiple times you should change the seed each time (for example, you could increment the seed each time you call the function)

In [22]:
mydata = []
#  make 10 signal events with a signalToBackground of 1 using random seed 123
mydata=makeData(1,20,123)
print("number of total data events: ",len(mydata))
print("data: ",mydata)

number of total data events:  115
data:  [11.047401505881462, 11.57102936217666, 10.430661187946646, 0.05376129148641384, 19.7669083856564, 18.106831513232198, 4.15271722392649, 5.849788255848496, 10.40020306144967, 18.038227453213413, 19.672617698234465, 5.150841283081662, 11.287180858495633, 10.579689779144333, 7.887401079055496, 14.621460716891141, 3.221380288584297, 12.013971356671798, 17.317289166065294, 19.67043218407111, 1.5873158075603144, 8.566945494018984, 4.0908571909285545, 9.012729810374696, 10.95527145257708, 1.8665342073964153, 5.937215509613589, 18.55168480304295, 11.380074628603907, 9.148239950472238, 8.762646787312146, 14.837243036840746, 0.9715806568853758, 14.17394790885492, 16.784866956101673, 3.3187576841390776, 15.619958759999147, 5.730732334582038, 6.129395066591146, 13.30522930699366, 2.227843432154315, 9.800887617242747, 17.757135853524453, 13.926225364708127, 8.806557533308181, 8.764287687544494, 15.30192190478613, 11.312840024517657, 1.6980832638363519, 11.6

Generate a sample of 1000 signal events with signalToBackground=0.5 and make a histogram of your results. Make sure that the number of bins in your histogram is small enough that you have at least 10 entries per bin (so that it is reasonable to do a binned fit to the histogram). To make life a bit easier, here is function you can use to make your histograms.  You are of course free to write your own function and not use this code.

In [23]:
#Import the pyplot module of matplotlib as "plt"
import matplotlib.pyplot as plt


#Makes a histogram filled with the random numbers we generate
def plot_histogram(samples,xtitle,ytitle, title, nbins, limits):
   
    #Plot the histogram of the sampled data with nbins and a nice color
    n, bins, patches =plt.hist(samples, bins=nbins, range=limits, color=(0,0.7,0.9))  #Set the color using (r,g,b) values or
                                                                  #  use a built-in matplotlib color""" 
    bincenters = 0.5*(bins[1:]+bins[:-1])
    errs = np.sqrt(n)

    plt.errorbar(bincenters, n, yerr=errs, fmt='none')
    #Add some axis labels and a descriptive title
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)

    #Get rid of the extra white space on the left/right edges (you can delete these two lines without a problem)
    xmin, xmax, ymin, ymax = plt.axis()
    plt.axis([limits[0],limits[1],ymin,ymax])

    #Not necessarily needed in a Jupyter notebook, but it doesn't hurt
    plt.show()
    return n, bins, patches

or alternatively using ROOT as follows:

In [93]:
#Import the pyplot module of matplotlib as "plt"
import math


#Makes a histogram filled with the random numbers we generate
def plot_histogram(samples,xtitle,ytitle, title, nbins, limits):
       
    #Create the histogram to contain the sampled data with nbins
    h_sampled = ROOT.TH1F("h_sampled", "Sampled data", nbins, limits[0], limits[1])
    #Add some axis labels and a descriptive title
    h_sampled.SetXTitle(xtitle)
    h_sampled.SetYTitle(ytitle)
    #Set a nice color
    h_sampled.SetLineColor(ROOT.kBlack)
    
    #Add data to the histogram
    for s in samples:
        h_sampled.Fill(s)
    
    #Draw the histogram
    c_sampled = ROOT.TCanvas("c_sampled")
    h_sampled.Draw("HIST E") #Draw histogram with error bars following Gaussian statistics
    
    #Adjust y-axis range
    h_sampled.GetYaxis().SetRangeUser(0, h_sampled.GetMaximum()*1.2)
    
    c_sampled.Draw()
    
    #return the canvas and histograms if more work is needed
    return c_sampled, h_sampled

In [ ]:
#Write your answer here

### 1b. 

Pretend this is real data.  Your goal is to find the best estimate of how many events are in a Gaussian peak with unknown mean and sigma and what the uncertainty on this estimate is.  In your fit, you can make the assumption that the background is flat (a zeroth order polynomial) but that you don't have a prediction for the background rate.  Use your favorite minimization package to fit the data.  Deterimine the best estimate of the number of events in signal and the uncertainty on that estimate.  (remember, that you must let the position and width of the Gaussian and the size of the background vary in your fit).

Hint: For examples of how to perform a least squared fit of a function to data see:
- https://github.com/berkeley-physics/Python-Tutorials/blob/master/3%20-%20Specific%20topics/Fitting.ipynb
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
- https://root.cern/doc/master/multifit_8py.html (ROOT, python)
- https://root.cern/doc/master/FittingDemo_8C.html (ROOT, more complete example in C++)

In [ ]:
#Write your answer here

### 1c. 

What is the $\chi^2$ per degree of freedom for your fit?  What does this number tell you about how well your fit describes the data?

In [ ]:
#Write your answer here

### 1d. 

If a signal of size $N_S^{meas}$ has a fitted uncertainty $\sigma_S$, the signficance of the measurement is defined to be
$$
S^{meas} \equiv \frac{N_S^{meas}}{\sigma_S}
$$
When the size of the data sample is large enough that Gaussian uncertainties are appropriate, a rule of thumb can be used to give a crude estimate of the expected significance $S^{expected}$ of the measurement. This predicted signficance depends on the number of signal events ($N_{S}$) and is the number of background events populating the region **beneath the signal** ($N_B$):
$$
S^{expected}  \approx  \frac{N_S}{N_S+N_B}
$$

Repeat the above exercise changing both the number of events in your signal and the signalToBackground ratio.  Plot the values of the measured significance  $S^{meas}$ 
obtained from your fits as a function of $\frac{N_S}{\sqrt{N_S+N_B}}$.  How do your results compare to the simple rule of thumb quoted above?

In [ ]:
#Write your answer here

## Question 2: Likelihood Fits, Statistical Methods (40 points)

### Learning objectives
In this question you will:

- Construct a likelihood function for a probability distribution with one free parameter
- Determine the best fit value of the parameter and estimate its uncertainty both by graphing the likelihood function and using a standard minimization package


Consider the problem of determining the lifetime of a species of particle that we can stop in our detector by observing its decays. Assume each time a particle stops, we set the stopping time to be $t=0$ and that we only observe decays that occur up to a time $T_{max}$ after the particle stops.  The distribution of measured times therefore follows the form:

\begin{eqnarray*}
R(t) & =  R_0 e^{-\Gamma t} & \qquad \qquad 0 \le t \le T_{max} \\
     & =  0 &\qquad \qquad {\rm otherwise}
\end{eqnarray*}

For this problem, we'll take as the true decay parameter $\Gamma=2\ \mathrm{sec}^{-1}$ and maximum time that we wait for a decay to be $T_{max}=3$ sec. We can imagine doing the experiment over many times (each experiment takes three seconds) to accumulate a lot of data.

### 2a. 

First, let's generate some fake data. Generate 1000 decay times that follow the formula above.  (Hint:  use numpy.random.exponential or ROOT.gRandom.Exp and reject events with decay times larger than $T_{max}$.  Verify that your event generation looks reasonable by making a histogram of the decay times.

In [14]:
#Write import math
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def makeData( Gamma, Tmax, nDecays ):
    """Generates a dataset of decay times.  The distribution of events follows an exponential with 
    decay parameter Gamma, but where the decays are cut-off after time Tmax
    
    Parameters
    ==========
    Gamma : float
      decay parameter of the exponential distribution
      
    Tmax : float
      maximum decay time that can be generated in the dataset
      
    nDecays : int
      number of decay times to generate
      
    Returns
    =======
    decayTimes : array
      nDecays number of decay times generated according to the exponential distribution with decay
      parameter gamma and maximum decay time Tmax
    """
    
    # Make an array to hold the decay times
    decayTimes = [0.0 for i in range(nDecays)]
    
    '''Your code here'''
    
    return decayTimes

# Run your function and put plotting code here

or using ROOT as follows (reminder: do not execute both cells above, just either this or the one above depending on what you're planning to use and delete the other from your notebook to avoid mistakes)

In [18]:
#Write import math
import ROOT

def makeData( Gamma, Tmax, nDecays ):
    """Generates a dataset of decay times.  The distribution of events follows an exponential with 
    decay parameter Gamma, but where the decays are cut-off after time Tmax
    
    Parameters
    ==========
    Gamma : float
      decay parameter of the exponential distribution
      
    Tmax : float
      maximum decay time that can be generated in the dataset
      
    nDecays : int
      number of decay times to generate
      
    Returns
    =======
    decayTimes : array
      nDecays number of decay times generated according to the exponential distribution with decay
      parameter gamma and maximum decay time Tmax
    """
    
    # Make an array to hold the decay times
    decayTimes = [0.0 for i in range(nDecays)]
    
    '''Your code here'''
    
    return decayTimes

# Run your function and put plotting code here

### 2b. 

Calculate the negative log-likelihood function, $-\ln {\cal L}$, for your data.  Express your likelihood in terms of the free parameters, $\Gamma$.

Hint: your pdf for the expected distribution of decay times $f(t) $ is an exponental that only extends to $t_{max}$ so:
$$
\int_0^{t_{max}}f(t)dt = \int_0^{t_{max}} R_0 e^{-\Gamma t} dt = 1
$$

Write your answer here

### 2c. 

We will study the simulated data, pretending that we dont know what value of $\Gamma$ was used to generate it.  We want to find the best estimate of $\Gamma$ from the data. 

We saw in class that for high statistics $-2\ln {\cal L}$ is  distributed like a $\chi^2$ distribution and the uncertainty on the estimate of a parameter of the function can be obtained by finding how much the you can change the parameter to increase $-2\ln {\cal L}$ by 1. 
Write code to calculate the negative log-likelihood:
$$
- 2 \ln {\cal L} = -2 \sum_i \ln f(\Gamma, t_i)
$$
where the $t_i$ are the time values you generated.  Using this code, find the value of $-2\ln {\cal L}$ for $\Gamma - \Gamma_{true}$.

In [16]:
def minusloglikelihoodFn(Gamma, maxT, decayTimes):
    """calculates the -ln(Likelihood) for the decayTimes for specified values of maxT and Gamma
    
    Parameters
    ==========
    Gamma : float
      hypothesis for lifetime
      
    Tmax : float
      maximum time for observation
      
    nDecays : array
      a dataset of decay times generated according to the distribution described above
      
    Returns
    =======
    minusLogLikelihood : float
      -ln(Likelihood) given the hypothesized Gamma and maxT for the input data
    """  
    minusLogLikelihood = 0.0
    
    '''Your code here'''

    return minusLogLikelihood

### 2d. 

There are lots of algorithms for finding the  minimum of a non-linear function such as our negative log-likelihood,  but we won't bother to use any of these algorithms for yet.  Instead, we will explore the minimum by inspecting the behavour of the function. Plot the value of $-\ln {\cal L}$ you obtain from your simulated data as you vary $\Gamma$ in the region  of the true answer ($\Gamma=2$).  How close is the $\Gamma $ that gives minimum negative log-likelihood  to the true value of $\Gamma$?  Estimate the uncertainty on your estimate of $\Gamma$ by finding the values corresponding to an increase of ${\cal L}$ of 1.0

In [18]:
ngrid=100 # How many points to scan
G = np.arange(1.75,2.25,0.5/ngrid)  #The gamma values to scan
LLG = np.zeros(ngrid)               #The likelihood for these values

# Your code here

or using ROOT as follows

In [26]:
ngrid=100 # How many points to scan
G_min = 1.75
G_max = 2.25
G_step = (G_max-G_min)/ngrid

G = [] #The gamma values to scan
LLG = [0.0 for i in range(ngrid)] #The likelihood for these values
for i in range(ngrid):
    G.append(G_min + G_step*i) 

# Your code here

### 2e. 

To verify your estimate of the uncertainty on the measured value of $\Gamma$, generate 100 samples each with 1000 events.  Histogram the estimated $\Gamma$ for these samples and find the rms of the "measurements."  How does this rms compare to your answers above?

Note: while minimizing via a scan as above is instructive, here you can also try to use existing minimizers, e.g. scipy.optimize.minimize using the 'BFGS' method or ROOT.Math.Minimizer with e.g. Minuit2  (or just your custom code you wrote above).

In [ ]:
#Write your answer here

## Question 3: Accelerator performance (30 points)

### Learning objectives
In this question you will:

- familiarize with differences and design choices for colliders and fixed-target experiments

### 3a. 

Explicitly derive the expressions for the center-of-mass energy for the collider and fixed target caes not assuming that the masses of the colliding particles are small

Write your answer here...

### 3b. 

What approximations do you have to make to derive the simpler formulae usually quoted:
$$ E_{\textrm{cm}}^{\textrm{collider}} = \sqrt{4E_1E_2},\quad E_{cm}^{\textrm{fixded-target}}=\sqrt{2m_{\textrm{target}}E_{\textrm{beam}}} $$

Write your answer here...

### 3c. 

The Babar experiment (which ran at SLAC until Spring 2008) and the upgraded Belle experiment at KEK in Japan study B mesons produced in $\Upsilon(4S)$ decays through the process $e^+e^− \to \Upsilon(4S) \to BB$.
Both accelerators were designed to operate at a center-of-mass energy of $10.56$ GeV (corresponding to the $\Upsilon(4S)$ mass). Both accelerators are “asymmetric colliders,” where the energies of the $e^+$ and $e^−$ beams are not the same. This asymmetry means that the center-of-mass is boosted along the beam direction and therefore the produced B mesons are moving, making it possible to measure their lifetime by looking at the distance they travel before decaying.

(i) For a boost of $\beta\gamma =$ 0.56 along the $e^+$ direction of motion, determine the required energies of the $e^+$ and $e^−$

(ii) Determine the average distance between the B production point and its decay point in the LAB. You can find the value of the B meson lifetime on the PDG website, [pdg.lbl.gov](https://pdg.lbl.gov), using the pdgLive tool. The charged and neutral B mesons have almost the same lifetime. Use the $B^0$ lifetime for this problem. Also, note that in the center-of-mass energy of the B-factories is very close to twice the B mass. You may therefore make the approximation that the B mesons are produced at rest in the center-of-mass.


Write your answer here...